In [1]:
"""Specification Test: Income segmentation: Medium income.

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("../Data/swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ) + (  INCOME != 2 ))>0
database.remove(exclude)

# Estimation of the model
logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "SpecTest_SM_medium_income"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	0.354
ASC_SM=	0.757
B_CAR_COST=	-0.0066
B_CAR_TIME=	-0.0113
B_FIRST=	0.421
B_GA=	0.466
B_HE=	-0.00844
B_SENIOR=	-2.29
B_SM_COST=	-0.0156
B_SM_TIME=	-0.00309
B_TRAIN_COST=	-0.0313
B_TRAIN_TIME=	-0.00212
                 Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR       0.354133  0.262802   1.347528  1.778103e-01      0.273216   
ASC_SM        0.756981  0.214001   3.537282  4.042680e-04      0.218249   
B_CAR_COST   -0.006597  0.002020  -3.266631  1.088353e-03      0.002149   
B_CAR_TIME   -0.011275  0.001410  -7.998197  1.332268e-15      0.001629   
B_FIRST       0.421490  0.123730   3.406518  6.579710e-04      0.139722   
B_GA          0.465914  0.371412   1.254440  2.096822e-01      0.400877   
B_HE         -0.008437  0.002054  -4.108082  3.989589e-05      0.002061   
B_SENIOR     -2.286468  0.248114  -9.215406  0.000000e+00      0.236088   
B_SM_COST    -0.015596  0.001245 -12.528195  0.000000e+00      0.002345   
B_SM_TIME    -0.003085  0.000921  -3.34